# Lesson 3. Introduction to SimPy

### SA421 Fall 2015

## What is SimPy?

* SimPy is a **process-based** discrete-event simulation language based on Python.


* An **event-based simulation** models the operation of a system by describing what happens to the system as it encounters entities.
    - e.g. how does the number of customers in the queue change when a customer arrives?


* A **process-based simulation** models the operation of a system by describing what happens to each entity as it encounters the system.
    - e.g. when a customer arrives, where does she go?


* Each has its advantages and disadvantages, for example:
    - The event-based view is very flexible and appealing from a mathematical perspective.
    - The process-based view is usually easier to use and manage for typical simulation scenarios.


* <span style="color:#a00000;">**Note: we will be using SimPy 2.3 in this course, not SimPy 3.0!**</span>

## The Fantastic Dan Problem in SimPy

**Problem.** Customers visit the neighborhood hair stylist Fantastic Dan for haircuts. The customer interarrival time is exponentially distributed with mean 20 minutes. Each haircut takes Fantastic Dan anywhere from 15 to 25 minutes, uniformly distributed. This time also includes the initial greetings and the transaction of money at the end of the haircut.

1. Simulate 1 day of Dan’s operations. Assume Dan works continuously for 6 hours a day.
2. How many customers enter Dan's shop per day?
3. How many customers does Dan serve per day?
4. What is the average number of customers waiting to get a haircut? What is the maximum? 
5. What is the average time spent by a customer in the queue? What is the maximum?

### Some preliminary steps

* There are <span style="color:#a00000;">a lot</span> of **libraries** (i.e. **packages**) you can use with Python
    * For examples, take a look at the [Python Package Index (PyPI)](https://pypi.python.org/pypi)


* Most libraries aren't available by default.


* To access functions in these libraries, we need to use `from` and `import`.


* These preliminary statements will often be at the top of our notebooks.


* `SimPy.Simulation` is the SimPy library. For the sake of simplicity, we'll just import everything (`*`) from it. 


* `numpy.random` is a NumPy library for random sampling. Check out the documentation [here](http://docs.scipy.org/doc/numpy/reference/routines.random.html). 

In [1]:
##### Setup #####
# Import everything from SimPy
from SimPy.Simulation import *

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, uniform, exponential

* A basic SimPy **model** consists of:
    - Parameters
    - Processes
    - Resources


* Once we have the model in hand, we can run **experiments** with the model to answer the questions above.

### Parameters

* As good practice, we create **parameters** for values in our model that we might want to change later on. 


* For the Fantastic Dan problem, we can create parameters for
    * the mean interarrival time
    * the minimum service time
    * the maximum service time
    * the number of servers (barbers)

In [2]:
##### Parameters #####
class P:
    # Customers arrive at the entrance with exponentially distributed
    # interarrival times with mean 20
    interarrivalTimeMean = 20
    
    # Service times are uniformly distributed between 15 and 25
    serviceTimeMin = 15
    serviceTimeMax = 25
    
    # One server: Fantastic Dan works by himself
    nServers = 1
    
    # Shop is open for 6 continuous hours
    simulationTimeMax =  6 * 60

* By putting all the parameters in one class `P`, we can access the parameters with a common syntax: for example, `P.interarrivalTimeMean`, `P.serviceTimeMax` and so on.

### Processes

* A **process** is an active component of a simulation model.
    - e.g. a customer that moves between the queue and the server in the shop.
    - A process is sometimes referred to as an **entity**.


* To define a process, we need to describe its behavior.
    - e.g. what does a customer do when she enters the shop?

#### The customer process

* Let's start by defining a process that defines a customer's behavior in this system.


* A customer's behavior:
    * Customer arrives from entrance and joins queue.
    * Customer is released from queue and starts haircut.
    * Customer finishes haircut, and leaves.

In [3]:
##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        print("Time {1}: {0} arrives and joins queue".format(self.name, now()))
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        print("Time {1}: {0} begins service".format(self.name, now()))
        serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        print("Time {1}: {0} ends service and leaves".format(self.name, now()))
        yield release, self, R.server

* `now()` gives the current time in the simulation.


* `yield request` has the entity request a unit of a **resource**. 
    * More on this in a second.


* `yield hold` has the entity pause for the specified amount of time.
    * This is often used to simulate service time.


* `uniform` is a random sampling function defined in the `numpy.random` module: 
    * `uniform(low, high)` generates a random sample from a uniform distribution on `[low, high]`


* `self` refers to the generic entity we're defining.
    * `self.name` refers to the name of the entity.
    * We'll see how this name gets defined shortly.

#### The entrance process

* We need to generate customers to enter the simulation model. 


* We can view the shop entrance as a process that generates customers.

In [4]:
# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

* A process is passive when it is first created.


* To activate a process's behavior, use `activate(entity, entity.behavior())`.


* `exponential` is a random sampling function defined in the `numpy.random` module.
    * `exponential(scale)` generates a random sample from an exponential distribution with mean `scale`.
    * Check out the documentation [here](http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.exponential.html#numpy.random.exponential).

### Resources

* A **resource** is something that processes an entity.


* In SimPy, an entity requests a unit of a resource, holds it for some time, and then releases it.


* A **queue** for the resource will form if the number of requests exceeds the number of resource units available.


* This queue is automatically maintained by the resource in SimPy.


* For now, we just create placeholders for all the resources we will need &ndash; in this case, a server (barber).


* We will define the resource later.

In [5]:
##### Resources #####
class R:
    # Server
    server = None

* As with the parameters, we put all the resources in 1 class, so that we can access them in a consistent way, e.g. `R.server`, etc. 

### Running the simulation model

* With almost all of the simulation model components in place, we are ready to run the simulation model.


* To do so, let's create a function that runs the model.


* In particular, we need to 
    - initialize SimPy and the random number generator,
    - define the server resource,
    - activate the entrance process, and
    - start the simulation.

In [6]:
##### Model #####
def model():
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator (more on this later in the semester)
    seed(123)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* `Resource(capacity = n)` defines a resource that can process $n$ entities simultaneously.
    - Note that resource definitions must occur *after* the `initialize()` statement.

Now, run the simulation model!

In [7]:
##### Experiment #####
model()

Time 23.84544286962211: Customer 0 arrives and joins queue
Time 23.84544286962211: Customer 0 begins service
Time 30.58679252313273: Customer 1 arrives and joins queue
Time 41.113957405264145: Customer 0 ends service and leaves
Time 41.113957405264145: Customer 1 begins service
Time 46.615466159426475: Customer 2 arrives and joins queue
Time 57.617416875006015: Customer 3 arrives and joins queue
Time 63.308647103119775: Customer 1 ends service and leaves
Time 63.308647103119775: Customer 2 begins service
Time 85.1569444889684: Customer 2 ends service and leaves
Time 85.1569444889684: Customer 3 begins service
Time 104.96626350381202: Customer 3 ends service and leaves
Time 136.63705825130037: Customer 4 arrives and joins queue
Time 136.63705825130037: Customer 4 begins service
Time 146.5925322953354: Customer 5 arrives and joins queue
Time 155.06883841280907: Customer 4 ends service and leaves
Time 155.06883841280907: Customer 5 begins service
Time 172.70893023548854: Customer 6 arrive

So far, based on our 1 simulation run, we can answer the following parts of the Fantastic Dan problem:

* How many customers enter Dan's shop per day?

* How many customers does Dan serve per day?

22 customers enter the shop (Customers 0 to 21), and 15 get completely served (Customers 0 to 14). 

## SimPy resources

* Links to the official SimPy documentation have been posted on the course webpage.
    - The SimPy documentation has some nice tutorials.


* In addition, links to the NumPy, SciPy, and Matplotlib documentation have been posted on the course webpage. We will use these packages often, for things like 
    * random sampling
    * statistical testing
    * graphs and plots

## With a neighbor...

Here is the entire Fantastic Dan simulation model in 1 cell, for your convenience.

In [8]:
##### Setup #####
# Import all simulation functions from SimPy
from SimPy.Simulation import *

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, uniform, exponential


##### Parameters #####
class P:
    # Customers arrive at the entrance with exponentially distributed
    # interarrival times with mean 20
    interarrivalTimeMean = 20
    
    # Service times are uniformly distributed between 15 and 25
    serviceTimeMin = 15
    serviceTimeMax = 25
    
    # One server: Fantastic Dan works by himself
    nServers = 1
    
    # Shop is open for 6 continuous hours
    simulationTimeMax =  6 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        print("Time {1}: {0} arrives and joins queue".format(self.name, now()))
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        print("Time {1}: {0} begins service".format(self.name, now()))
        serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        print("Time {1}: {0} ends service and leaves".format(self.name, now()))
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

##### Resources #####
class R:
    # Server
    server = None


##### Model #####
def model():
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator (more on this later in the semester)
    seed(123)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

    
##### Experiment #####
model()

Time 23.84544286962211: Customer 0 arrives and joins queue
Time 23.84544286962211: Customer 0 begins service
Time 30.58679252313273: Customer 1 arrives and joins queue
Time 41.113957405264145: Customer 0 ends service and leaves
Time 41.113957405264145: Customer 1 begins service
Time 46.615466159426475: Customer 2 arrives and joins queue
Time 57.617416875006015: Customer 3 arrives and joins queue
Time 63.308647103119775: Customer 1 ends service and leaves
Time 63.308647103119775: Customer 2 begins service
Time 85.1569444889684: Customer 2 ends service and leaves
Time 85.1569444889684: Customer 3 begins service
Time 104.96626350381202: Customer 3 ends service and leaves
Time 136.63705825130037: Customer 4 arrives and joins queue
Time 136.63705825130037: Customer 4 begins service
Time 146.5925322953354: Customer 5 arrives and joins queue
Time 155.06883841280907: Customer 4 ends service and leaves
Time 155.06883841280907: Customer 5 begins service
Time 172.70893023548854: Customer 6 arrive

See if you can figure out how to make the following modifications. You may need to look at the documentation, especially for different random sampling functions. 

For each modification, <span style="color:#a00000;">start by copying and pasting the entire simulation model into a new cell</span>. Make the changes, and run the model. How do the number of customers that enter Dan's shop and the number of customers Dan serves per day change?

**Modification.** The time between customer arrivals is still exponentially distributed, but now with a mean of 30 minutes. The time for each haircut is still uniformly distributed, but now between 25 and 35 minutes.

To make these changes, we need to change the parameters for the mean interarrival time and the minimum and maximum service time, like so:

```python
##### Parameters #####
class P:
    # Customers arrive at the entrance with exponentially distributed
    # interarrival times with mean 30
    interarrivalTimeMean = 30
    
    # Service times are uniformly distributed between 25 and 35
    serviceTimeMin = 25
    serviceTimeMax = 35
    ...
```

In [9]:
##### Setup #####
# Import all simulation functions from SimPy
from SimPy.Simulation import *

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, uniform, exponential


##### Parameters #####
class P:
    # Customers arrive at the entrance with exponentially distributed
    # interarrival times with mean 30
    interarrivalTimeMean = 30
    
    # Service times are uniformly distributed between 25 and 35
    serviceTimeMin = 25
    serviceTimeMax = 35
    
    # One server: Fantastic Dan works by himself
    nServers = 1
    
    # Shop is open for 6 continuous hours
    simulationTimeMax =  6 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        print("Time {1}: {0} arrives and joins queue".format(self.name, now()))
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        print("Time {1}: {0} begins service".format(self.name, now()))
        serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        print("Time {1}: {0} ends service and leaves".format(self.name, now()))
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

##### Resources #####
class R:
    # Server
    server = None


##### Model #####
def model():
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator (more on this later in the semester)
    seed(123)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)


##### Experiment #####
model()

Time 35.76816430443317: Customer 0 arrives and joins queue
Time 35.76816430443317: Customer 0 begins service
Time 45.8801887846991: Customer 1 arrives and joins queue
Time 63.0366788400752: Customer 0 ends service and leaves
Time 63.0366788400752: Customer 1 begins service
Time 69.92319923913972: Customer 2 arrives and joins queue
Time 86.42612531250903: Customer 3 arrives and joins queue
Time 95.23136853793082: Customer 1 ends service and leaves
Time 95.23136853793082: Customer 2 begins service
Time 127.07966592377946: Customer 2 ends service and leaves
Time 127.07966592377946: Customer 3 begins service
Time 156.88898493862308: Customer 3 ends service and leaves
Time 204.95558737695058: Customer 4 arrives and joins queue
Time 204.95558737695058: Customer 4 begins service
Time 219.8887984430031: Customer 5 arrives and joins queue
Time 233.38736753845927: Customer 4 ends service and leaves
Time 233.38736753845927: Customer 5 begins service
Time 259.06339535323286: Customer 6 arrives and

With these modifications, 13 customers enter the shop (Customers 0 to 12), and 9 get completely served (Customers 0 to 8).

**Modification.** The haircut time instead follows a Gamma distribution with shape parameter 3 and scale parameter 10. [Hint.](http://docs.scipy.org/doc/numpy/reference/routines.random.html) Also, remember to `import` the function you want to use.

To make this change, we need to use the `gamma` function from `numpy.random`. So, first we modify the `from/import` statement in the setup:

```python
# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, uniform, exponential, gamma
```

Next, we need to change the parameters to reflect the change in the service time distribution, like so:

```python
##### Parameters #####
class P:
    ...
    # Service times follow a Gamma distribution with shape 3 and scale 10
    serviceTimeShape = 3
    serviceTimeScale = 10    
    ...
```

Finally, we need to change the definition of the customer process to reflect this change in the service time distribution.

```python
##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        ...
        # Customer is released from queue and starts service
        print("Time {1}: {0} begins service".format(self.name, now()))
        serviceTime = gamma(shape = P.serviceTimeShape, scale = P.serviceTimeScale)
        yield hold, self, serviceTime
        ...
```

In [10]:
##### Setup #####
# Import all simulation functions from SimPy
from SimPy.Simulation import *

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, uniform, exponential, gamma


##### Parameters #####
class P:
    # Customers arrive at the entrance with exponentially distributed
    # interarrival times with mean 20
    interarrivalTimeMean = 20
    
    # Service times follow a Gamma distribution with shape 3 and scale 10
    serviceTimeShape = 3
    serviceTimeScale = 10
    
    # One server: Fantastic Dan works by himself
    nServers = 1
    
    # Shop is open for 6 continuous hours
    simulationTimeMax =  6 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        print("Time {1}: {0} arrives and joins queue".format(self.name, now()))
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        print("Time {1}: {0} begins service".format(self.name, now()))
        serviceTime = gamma(shape = P.serviceTimeShape, scale = P.serviceTimeScale)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        print("Time {1}: {0} ends service and leaves".format(self.name, now()))
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

##### Resources #####
class R:
    # Server
    server = None


##### Model #####
def model():
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator (more on this later in the semester)
    seed(123)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

    
##### Experiment #####
model()

Time 23.84544286962211: Customer 0 arrives and joins queue
Time 23.84544286962211: Customer 0 begins service
Time 30.58679252313273: Customer 1 arrives and joins queue
Time 41.58874323871227: Customer 2 arrives and joins queue
Time 55.40519122403728: Customer 0 ends service and leaves
Time 55.40519122403728: Customer 1 begins service
Time 64.26210461737892: Customer 1 ends service and leaves
Time 64.26210461737892: Customer 2 begins service
Time 67.68943131453943: Customer 2 ends service and leaves
Time 120.60838461500663: Customer 3 arrives and joins queue
Time 120.60838461500663: Customer 3 begins service
Time 140.8662614272006: Customer 3 ends service and leaves
Time 146.72478255515978: Customer 4 arrives and joins queue
Time 146.72478255515978: Customer 4 begins service
Time 147.95543853980772: Customer 5 arrives and joins queue
Time 151.81383105650724: Customer 6 arrives and joins queue
Time 166.98040784308287: Customer 7 arrives and joins queue
Time 182.15878078846833: Customer 8

With these changes, 21 customers enter the shop (Customers 0 to 20), and 8 get served completely (Customers 0 to 7).

**Modification**. Customers arrive at Dan's shop in pairs. The interarrival time for pairs of customers is still exponentially distributed with a mean of 20 minutes. 

Here, we need to change how customers are generated at the entrance: in particular, instead of initializing and activating just 1 customer at a time, we can initialize and activate 2 customers at a time:

```python
# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1
            
            # At the same time, create another new customer using the template 
            # defined in the Customer class (since they come in pairs)
            c2 = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c2, c2.behavior())

            # Count this new customer
            nCustomers += 1
```

In [11]:
##### Setup #####
# Friendly floating-point division
from __future__ import division

# Import all simulation functions from SimPy
from SimPy.Simulation import *

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, uniform, exponential


##### Parameters #####
class P:
    # Customers arrive at the entrance with exponentially distributed
    # interarrival times with mean 20
    interarrivalTimeMean = 20
    
    # Service times are uniformly distributed between 15 and 25
    serviceTimeMin = 15
    serviceTimeMax = 25
    
    # One server: Fantastic Dan works by himself
    nServers = 1
    
    # Shop is open for 6 continuous hours
    simulationTimeMax =  6 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        print("Time {1}: {0} arrives and joins queue".format(self.name, now()))
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        print("Time {1}: {0} begins service".format(self.name, now()))
        serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        print("Time {1}: {0} ends service and leaves".format(self.name, now()))
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1
            
            # At the same time, create another new customer using the template 
            # defined in the Customer class (since they come in pairs)
            c2 = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c2, c2.behavior())

            # Count this new customer
            nCustomers += 1

##### Resources #####
class R:
    # Server
    server = None


##### Model #####
def model():
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator (more on this later in the semester)
    seed(123)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

    
##### Experiment #####
model()

Time 23.84544286962211: Customer 0 arrives and joins queue
Time 23.84544286962211: Customer 0 begins service
Time 23.84544286962211: Customer 1 arrives and joins queue
Time 30.58679252313273: Customer 2 arrives and joins queue
Time 30.58679252313273: Customer 3 arrives and joins queue
Time 41.113957405264145: Customer 0 ends service and leaves
Time 41.113957405264145: Customer 1 begins service
Time 46.615466159426475: Customer 4 arrives and joins queue
Time 46.615466159426475: Customer 5 arrives and joins queue
Time 57.617416875006015: Customer 6 arrives and joins queue
Time 57.617416875006015: Customer 7 arrives and joins queue
Time 63.308647103119775: Customer 1 ends service and leaves
Time 63.308647103119775: Customer 2 begins service
Time 85.1569444889684: Customer 2 ends service and leaves
Time 85.1569444889684: Customer 3 begins service
Time 104.96626350381202: Customer 3 ends service and leaves
Time 104.96626350381202: Customer 4 begins service
Time 123.88743868575352: Customer 

In this case, 36 customers enter the shop (Customers 0 to 35), and 17 get served completely (Customers 0 to 16).